### Set GPU

In [34]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

## Set Dataset Name

In [35]:
dataset_name = 'CIFAR10'
# dataset_name = 'CIFAR100'
# dataset_name = 'MNIST'
# dataset_name = 'TINYIMAGENET'
# dataset_name = 'IMBALANCED_CIFAR10'

### Run All Now

In [36]:
# from models.resnet_stl import resnet18
import torch
import numpy as np
from tqdm import tqdm

from models.resnet_cifar import resnet18
from utils.memory import MemoryBank
from utils.train_utils import simclr_train
from utils.utils import fill_memory_bank
from utils.config import create_config
from utils.common_config import get_model, get_train_dataset, get_val_transformations, get_train_dataloader
from utils.evaluate_utils import hungarian_evaluate2, scan_evaluate, get_predictions

In [37]:
output_folder = '../results/'
if dataset_name == "CIFAR10":
    output_folder += 'cifar-10/'
    config_exp_path = './configs/scan/scan_cifar10.yml'
    cfg_path = 'configs/CIFAR10_RESNET18.yaml'
elif dataset_name == "CIFAR100":
    output_folder += 'cifar-20/'
    config_exp_path = './configs/scan/scan_cifar20.yml'
    cfg_path = 'configs/CIFAR100_RESNET18.yaml'
elif dataset_name == "MNIST":
    output_folder += 'mnist/'
    config_exp_path = './configs/scan/scan_mnist.yml'
    cfg_path = 'configs/MNIST_RESNET18.yaml'
elif dataset_name == "TINYIMAGENET":
    output_folder += 'tinyimagenet/'
    config_exp_path = './configs/scan/scan_tinyimagenet.yml'
    cfg_path = 'configs/TINYIMAGENET_RESNET18.yaml'
elif dataset_name == 'IMBALANCED_CIFAR10':
    output_folder += 'imbalanced-cifar-10/'
    config_exp_path = './configs/scan/scan_cifar10_im.yml'
    cfg_path = 'configs/CIFAR10_RESNET18.yaml'
    
path_to_model = output_folder + 'scan/model.pth.tar'

state_dict = temp = torch.load(path_to_model)

In [38]:
import argparse

config_env_path = './configs/env.yml'
config = p = create_config(config_env_path, config_exp_path)

In [39]:
model = get_model(p)
model.load_state_dict(temp['model'])
model.eval()
model.cuda()

ClusteringModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [40]:
train_data = get_train_dataset(p, get_val_transformations(p),
                                        split='train',to_augmented_dataset=False) 
dataloader = get_train_dataloader(p, train_data)

Files already downloaded and verified


In [42]:
head = state_dict['head'] if config['setup'] == 'scan' else 0
predictions, features = get_predictions(config, dataloader, model, return_features=True)

In [79]:
@torch.no_grad()
def get_diff_norm(config, predictions, features, model, topk=10):
    import torch.nn.functional as F

    # Get topk most certain indices and pred labels
    print('Get topk')
    probs = predictions['probabilities']
    n_classes = probs.shape[1]
    dims = features.shape[1]
    max_probs, pred_labels = torch.max(probs, dim = 1)
    indices = torch.zeros((n_classes, topk))
    for pred_id in range(n_classes):
        probs_copy = max_probs.clone()
        mask_out = ~(pred_labels == pred_id)
        probs_copy[mask_out] = -1
        conf_vals, conf_idx = torch.topk(probs_copy, k = topk, largest = True, sorted = True)
        indices[pred_id, :] = conf_idx

    # Get corresponding features
    selected_features = torch.index_select(features, dim=0, index=indices.view(-1).long())
    selected_features = selected_features.unsqueeze(1).view(n_classes, -1, dims)

    # Get mean feature per class
    mean_features = torch.mean(selected_features, dim=1)
    # Get distance to mean feature
    mean_feature_selected = torch.index_select(mean_features, dim=0,index=pred_labels.view(-1).long())
    diff_features = features - mean_feature_selected
    diff_norm = torch.norm(diff_features, 2, dim=1)

    return diff_norm


In [80]:
head = state_dict['head'] if config['setup'] == 'scan' else 0
diff_norm = get_diff_norm(config, predictions[head], features, model, topk=300)


Get topk


In [83]:
diff_norm = diff_norm.cpu().numpy()
reverse_diff_norm = -diff_norm

In [93]:
np.save(f'{output_folder}/{dataset_name}_diff_norm.npy', diff_norm)
np.save(f'{output_folder}/{dataset_name}_reverse_diff_norm.npy', reverse_diff_norm)
np.save(f'{output_folder}/{dataset_name}_cluster_ids.npy', pred_labels)

In [92]:
pred_labels

tensor([1, 0, 5,  ..., 6, 2, 2])

In [85]:
diff_norm

array([8.252473, 6.36307 , 8.988281, ..., 7.013616, 4.261198, 7.480796],
      dtype=float32)

In [87]:
topk = 300

# Get topk most certain indices and pred labels
print('Get topk')
probs = predictions[head]['probabilities']
n_classes = probs.shape[1]
dims = features.shape[1]
max_probs, pred_labels = torch.max(probs, dim = 1)
indices = torch.zeros((n_classes, topk))
for pred_id in range(n_classes):
    probs_copy = max_probs.clone()
    mask_out = ~(pred_labels == pred_id)
    probs_copy[mask_out] = -1
    conf_vals, conf_idx = torch.topk(probs_copy, k = topk, largest = True, sorted = True)
    indices[pred_id, :] = conf_idx

# Get corresponding features
selected_features = torch.index_select(features, dim=0, index=indices.view(-1).long())
selected_features = selected_features.unsqueeze(1).view(n_classes, -1, dims)

# Get mean feature per class
mean_features = torch.mean(selected_features, dim=1)
# Get distance to mean feature
mean_feature_selected = torch.index_select(mean_features, dim=0,index=pred_labels.view(-1).long())
diff_features = features - mean_feature_selected
diff_norm = torch.norm(diff_features, 2, dim=1)

Get topk


In [90]:
selected_1k = torch.argsort(diff_norm)[:1000]
np.bincount(pred_labels[selected_1k])

array([ 77, 246,   2,  84,  22,  28, 146,  31, 306,  58])

In [91]:
selected_reverse_1k = torch.argsort(diff_norm)[-1000:]
np.bincount(pred_labels[selected_reverse_1k])

array([  0,   0,  15,  13, 426, 134,   0,  15,  22, 375])

In [96]:
loss_expectation = np.load(f'{output_folder}/{dataset_name}_SimCLR_losses_10_temp_0.3.npy')

In [104]:
losses_norm = (loss_expectation - np.mean(loss_expectation)) / np.std(loss_expectation)
diff_norm_norm = (diff_norm.numpy() - np.mean(diff_norm.numpy())) / np.std(diff_norm.numpy())

In [107]:
loss_weight = 0.5
near_weight = 1 - loss_weight
overall_score = loss_weight * losses_norm - near_weight * diff_norm_norm

In [109]:
overall_score

array([-1.00948315, -0.29201259, -0.43403473, ...,  0.50620154,
        0.02379469, -0.22676085])

In [108]:
np.save(f'{output_folder}/{dataset_name}_SCAN_0.3_overall.npy', overall_score)

In [111]:
selected_overall_1k = np.argsort(overall_score)[:1000]
np.bincount(pred_labels[selected_overall_1k])

array([ 50,  40,  51,  83, 309, 157,  25,  46,  49, 190])

### Change batch size if you run into out of memory error 

In [7]:
from pycls.datasets.data import Data
from pycls.config import cfg
cfg.merge_from_file(cfg_path)
cfg.DATASET.NAME = dataset_name
data_obj = Data(cfg)

train_data, train_size = data_obj.getDataset(save_dir='../data', isTrain=True, isDownload=True)
trainSet = [i for i in range(train_size)]
trainSet = np.array(trainSet, dtype=np.ndarray)
train_dataloader = data_obj.getSequentialDataLoader(indexes=trainSet, batch_size=256, data=train_data)

test_data, test_size = data_obj.getDataset(save_dir='../data', isTrain=False, isDownload=True)
test_dataloader = data_obj.getTestLoader(data=test_data, test_batch_size=cfg.TRAIN.BATCH_SIZE, seed_id=cfg.RNG_SEED)

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Train Mode: Contain 13996 images
Files already downloaded and verified
Test Mode: Contain 10000 images


In [8]:
import torch.nn.functional as F

@torch.no_grad()
def get_predictions(p, dataloader, model, return_features=False):
    # Make predictions on a dataset with neighbors
    model.eval()
    predictions = [[] for _ in range(p['num_heads'])]
    probs = [[] for _ in range(p['num_heads'])]
    targets = []
    if return_features:
        ft_dim = get_feature_dimensions_backbone(p)
        features = torch.zeros((len(dataloader.sampler), ft_dim)).cuda()
    
    key_ = 'image'

    ptr = 0
    for row in tqdm(dataloader, desc="Extracting Self Label Predictions"):
#         images = row['image']
#         lbl = row['target']
        images, lbl = row
        images = images.cuda()
        output = model(images, forward_pass='default')
        for i, output_i in enumerate(output):
            predictions[i].append(torch.argmax(output_i, dim=1))
        targets.append(lbl)

    predictions = [torch.cat(pred_, dim=0) for pred_ in predictions]
    targets = torch.cat(targets, dim=0)

    out = [{'predictions': pred_, 'targets': targets} for pred_, prob_ in zip(predictions, probs)]

    if return_features:
        return out, features.cpu()
    else:
        return out

In [9]:
# from utils.evaluate_utils import get_predictions

In [10]:
predictions = get_predictions(p, train_dataloader, model)

Extracting Self Label Predictions: 100%|██████████| 55/55 [00:04<00:00, 13.52it/s]


#### Note: Stats are irrelevant for CIFAR100

In [11]:
clustering_stats = hungarian_evaluate2(0, predictions, 
                                class_names=train_data.classes,
                                compute_confusion_matrix=False,
                                confusion_matrix_file=os.path.join('confusion_matrix.png'))

In [12]:
clustering_stats

{'ACC': 0.29351243212346384,
 'ARI': -0.012018654655578461,
 'NMI': 0.022166544097787316,
 'hungarian_match': [(0, 5),
  (1, 7),
  (2, 2),
  (3, 8),
  (4, 1),
  (5, 4),
  (6, 6),
  (7, 9),
  (8, 3),
  (9, 0)]}

In [13]:
predictions[0]['predictions'].cpu()

tensor([9, 9, 9,  ..., 9, 8, 9])

In [14]:
np.save(f'{output_folder}/{dataset_name}_SCAN_cluster_ids.npy', predictions[0]['predictions'].cpu())